<a href="https://colab.research.google.com/github/franlin1860/llm/blob/main/advanced_text_to_sql_v20240909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflows for Advanced Text-to-SQL

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/workflow/advanced_text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this guide we show you how to setup a text-to-SQL workflow over your data with our [workflows](https://docs.llamaindex.ai/en/stable/module_guides/workflow/#workflows) syntax.

This gives you flexibility to enhance text-to-SQL with additional techniques. We show these in the below sections:
1. **Query-Time Table Retrieval**: Dynamically retrieve relevant tables in the text-to-SQL prompt.
2. **Query-Time Sample Row retrieval**: Embed/Index each row, and dynamically retrieve example rows for each table in the text-to-SQL prompt.

Our out-of-the box workflows include our `NLSQLTableQueryEngine` and `SQLTableRetrieverQueryEngine`. (if you want to check out our text-to-SQL guide using these modules, take a look [here](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo.html)). This guide implements an advanced version of those modules, giving you the utmost flexibility to apply this to your own setting.

**NOTE:** Any Text-to-SQL application should be aware that executing
arbitrary SQL queries can be a security risk. It is recommended to
take precautions as needed, such as using restricted roles, read-only
databases, sandboxing, etc.

# Prevent Disconnection

In [1]:
#@markdown <h3>← 输入了代码后运行以防止断开</h>
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);

# LLM Setup

In [2]:
!pip install llama-index
!pip install llama_index-llms-openai_like
!pip install llama_index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [3]:
import os
os.environ["ZHIPU_API_KEY"] = ""

In [4]:
import os
import logging
import sys
from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings, ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 配置日志
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# 定义DeepSpeed model
llm = OpenAILike(model="glm-4-flash",
                 api_base="https://open.bigmodel.cn/api/paas/v4/",
                 api_key=os.environ["ZHIPU_API_KEY"],
                 temperature=0.6,
                 is_chat_model=True)

# 配置环境
Settings.llm = llm

# 设置嵌入模型
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-zh-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 256

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Load and Ingest Data


### Load Data
We use the [WikiTableQuestions dataset](https://ppasupat.github.io/WikiTableQuestions/) (Pasupat and Liang 2015) as our test dataset.

We go through all the csv's in one folder, store each in a sqlite database (we will then build an object index over each table schema).

In [5]:
!wget "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip" -O data.zip
!unzip data.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: WikiTableQuestions/csv/204-csv/553.tsv  
  inflating: WikiTableQuestions/csv/204-csv/147.html  
  inflating: WikiTableQuestions/csv/204-csv/774.table  
  inflating: WikiTableQuestions/csv/204-csv/986.csv  
  inflating: WikiTableQuestions/csv/204-csv/53.table  
  inflating: WikiTableQuestions/csv/204-csv/644.table  
  inflating: WikiTableQuestions/csv/204-csv/897.csv  
  inflating: WikiTableQuestions/csv/204-csv/648.html  
  inflating: WikiTableQuestions/csv/204-csv/653.html  
  inflating: WikiTableQuestions/csv/204-csv/808.table  
  inflating: WikiTableQuestions/csv/204-csv/389.html  
  inflating: WikiTableQuestions/csv/204-csv/96.table  
  inflating: WikiTableQuestions/csv/204-csv/981.csv  
  inflating: WikiTableQuestions/csv/204-csv/868.tsv  
  inflating: WikiTableQuestions/csv/204-csv/337.html  
  inflating: WikiTableQuestions/csv/204-csv/929.html  
  inflating: WikiTableQuestions/csv/204-csv/648.tsv  
  inflating: WikiTableQuestions/csv/204-c

In [6]:
import pandas as pd
from pathlib import Path

data_dir = Path("./WikiTableQuestions/csv/200-csv")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: WikiTableQuestions/csv/200-csv/0.csv
processing file: WikiTableQuestions/csv/200-csv/1.csv
processing file: WikiTableQuestions/csv/200-csv/10.csv
processing file: WikiTableQuestions/csv/200-csv/11.csv
processing file: WikiTableQuestions/csv/200-csv/12.csv
processing file: WikiTableQuestions/csv/200-csv/14.csv
processing file: WikiTableQuestions/csv/200-csv/15.csv
Error parsing WikiTableQuestions/csv/200-csv/15.csv: Error tokenizing data. C error: Expected 4 fields in line 16, saw 5

processing file: WikiTableQuestions/csv/200-csv/17.csv
Error parsing WikiTableQuestions/csv/200-csv/17.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 7

processing file: WikiTableQuestions/csv/200-csv/18.csv
processing file: WikiTableQuestions/csv/200-csv/20.csv
processing file: WikiTableQuestions/csv/200-csv/22.csv
processing file: WikiTableQuestions/csv/200-csv/24.csv
processing file: WikiTableQuestions/csv/200-csv/25.csv
processing file: WikiTableQuestions/csv/200-

### Extract Table Name and Summary from each Table

Here we use gpt-4o-mini to extract a table name (with underscores) and summary from each table with our Pydantic program.

In [7]:
tableinfo_dir = "WikiTableQuestions_TableInfo"
!mkdir {tableinfo_dir}

In [9]:
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.llms import ChatMessage


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise.
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """
prompt_tmpl = ChatPromptTemplate(
    message_templates=[ChatMessage.from_str(prompt_str, role="user")]
)

In [11]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            llm = Settings.llm
            table_info = llm.structured_predict(
                TableInfo,
                prompt_tmpl,
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

Processed table: AlbumChartPositions
Processed table: FilmTitlesAndPerformances
Processed table: TrafficAccidentStats
Processed table: AwardsNomineesResults
Processed table: TheatreAwardsNominees
Processed table: BadBoyAlbums
Processed table: RadioStationFrequencyInfo
Processed table: MissingPersonsIncidents
Processed table: MusicChartPositions
Processed table: KodachromeFilmHistory
Processed table: BBC_Radio_Financials
Processed table: AirfieldLocationInfo
Processed table: VoterPartyBreakdown
Processed table: ClubPerformanceStats
Processed table: HorseRaceWinners
Processed table: GrammyAwardsNominees
Processed table: BoxingMatchResults
Processed table: FootballSeasonResults
Processed table: DistrictPopulationDensity
Processed table: VoterPartyBreakdown
Table name VoterPartyBreakdown already exists, trying again.
Processed table: VoterBreakdownByParty
Processed table: AcademyAwardActressNominations
Processed table: MPOfficeSuccessions
Processed table: MunicipalityBoundaryChanges
Proces

### Put Data in SQL Database

We use `sqlalchemy`, a popular SQL database toolkit, to load all the tables.

In [12]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re


# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


# engine = create_engine("sqlite:///:memory:")
engine = create_engine("sqlite:///wiki_table_questions.db")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

Creating table: AlbumChartPositions
Creating table: FilmTitlesAndPerformances
Creating table: TrafficAccidentStats
Creating table: AwardsNomineesResults
Creating table: TheatreAwardsNominees
Creating table: BadBoyAlbums
Creating table: RadioStationFrequencyInfo
Creating table: MissingPersonsIncidents
Creating table: MusicChartPositions
Creating table: KodachromeFilmHistory
Creating table: BBC_Radio_Financials
Creating table: AirfieldLocationInfo
Creating table: VoterPartyBreakdown
Creating table: ClubPerformanceStats
Creating table: HorseRaceWinners
Creating table: GrammyAwardsNominees
Creating table: BoxingMatchResults
Creating table: FootballSeasonResults
Creating table: DistrictPopulationDensity
Creating table: VoterBreakdownByParty
Creating table: AcademyAwardActressNominations
Creating table: MPOfficeSuccessions
Creating table: MunicipalityBoundaryChanges
Creating table: FIFA_World_Cup_Group_Stage_Results
Creating table: ProbabilityEncodingTable
Creating table: MonthlyClimateStats

In [ ]:
# # setup Arize Phoenix for logging/observability
# import phoenix as px
# import llama_index.core

# px.launch_app()
# llama_index.core.set_global_handler("arize_phoenix")

## Advanced Capability 1: Text-to-SQL with Query-Time Table Retrieval.

We now show you how to setup an e2e text-to-SQL with table retrieval.

### Define Modules

Here we define the core modules.
1. Object index + retriever to store table schemas
2. SQLDatabase object to connect to the above tables + SQLRetriever.
3. Text-to-SQL Prompt
4. Response synthesis Prompt
5. LLM

Object index, retriever, SQLDatabase

In [13]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

SQLRetriever + Table Parser

In [14]:
from llama_index.core.retrievers import SQLRetriever
from typing import List

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)

Text-to-SQL Prompt + Output Parser

In [15]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(chat_response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = chat_response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


Response Synthesis Prompt

In [16]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

### Define Workflow

Now that the components are in place, let's define the full workflow!

In [17]:
from llama_index.core.workflow import (
    Workflow,
    StartEvent,
    StopEvent,
    step,
    Context,
    Event,
)


class TableRetrieveEvent(Event):
    """Result of running table retrieval."""

    table_context_str: str
    query: str


class TextToSQLEvent(Event):
    """Text-to-SQL event."""

    sql: str
    query: str


class TextToSQLWorkflow1(Workflow):
    """Text-to-SQL Workflow that does query-time table retrieval."""

    def __init__(
        self,
        obj_retriever,
        text2sql_prompt,
        sql_retriever,
        response_synthesis_prompt,
        llm,
        *args,
        **kwargs
    ) -> None:
        """Init params."""
        super().__init__(*args, **kwargs)
        self.obj_retriever = obj_retriever
        self.text2sql_prompt = text2sql_prompt
        self.sql_retriever = sql_retriever
        self.response_synthesis_prompt = response_synthesis_prompt
        self.llm = Settings.llm

    @step
    def retrieve_tables(
        self, ctx: Context, ev: StartEvent
    ) -> TableRetrieveEvent:
        """Retrieve tables."""
        table_schema_objs = self.obj_retriever.retrieve(ev.query)
        table_context_str = get_table_context_str(table_schema_objs)
        return TableRetrieveEvent(
            table_context_str=table_context_str, query=ev.query
        )

    @step
    def generate_sql(
        self, ctx: Context, ev: TableRetrieveEvent
    ) -> TextToSQLEvent:
        """Generate SQL statement."""
        fmt_messages = self.text2sql_prompt.format_messages(
            query_str=ev.query, schema=ev.table_context_str
        )
        chat_response = self.llm.chat(fmt_messages)
        sql = parse_response_to_sql(chat_response)
        return TextToSQLEvent(sql=sql, query=ev.query)

    @step
    def generate_response(self, ctx: Context, ev: TextToSQLEvent) -> StopEvent:
        """Run SQL retrieval and generate response."""
        retrieved_rows = self.sql_retriever.retrieve(ev.sql)
        fmt_messages = self.response_synthesis_prompt.format_messages(
            sql_query=ev.sql,
            context_str=str(retrieved_rows),
            query_str=ev.query,
        )
        chat_response = llm.chat(fmt_messages)
        return StopEvent(result=chat_response)

### Visualize Workflow

A really nice property of workflows is that you can both visualize the execution graph as well as a trace of the most recent execution.

In [22]:
!pip install pyvis
!pip install llama-index-utils-workflow

In [23]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    TextToSQLWorkflow1, filename="text_to_sql_table_retrieval.html"
)

text_to_sql_table_retrieval.html


In [24]:
from IPython.display import display, HTML

# Read the contents of the HTML file
with open("text_to_sql_table_retrieval.html", "r") as file:
    html_content = file.read()

# Display the HTML content
display(HTML(html_content))

### Run Some Queries!

Now we're ready to run some queries across this entire workflow.

In [29]:
workflow = TextToSQLWorkflow1(
    obj_retriever,
    text2sql_prompt,
    sql_retriever,
    response_synthesis_prompt,
    llm,
    timeout=300,
    verbose=True,
)

In [26]:
response = await workflow.run(
    query="What was the year that The Notorious B.I.G was signed to Bad Boy?"
)
print(str(response))

Running step retrieve_tables
Step retrieve_tables produced event TableRetrieveEvent
Running step generate_sql
Step generate_sql produced event TextToSQLEvent
Running step generate_response
Step generate_response produced event StopEvent
assistant: The year that The Notorious B.I.G. was signed to Bad Boy Records was 1993.


In [30]:
response = await workflow.run(
    query="Who won best director in the 1972 academy awards"
)
print(str(response))

Running step retrieve_tables
Step retrieve_tables produced event TableRetrieveEvent
Running step generate_sql
Step generate_sql produced event TextToSQLEvent
Running step generate_response
Step generate_response produced event StopEvent
assistant: It appears there was no winner listed for the Best Director category in the 1972 Academy Awards based on the provided query results.


In [32]:
response = await workflow.run(query="What was the term of Pasquale Preziosa?")
print(str(response))

Running step retrieve_tables
Step retrieve_tables produced event TableRetrieveEvent
Running step generate_sql
Step generate_sql produced event TextToSQLEvent
Running step generate_response
Step generate_response produced event StopEvent
assistant: The query you've provided does not return any results. It appears that there are no records in the AcademyAwardActressNominations table that match all the specified criteria, which are:

- Film: 'The Passion of the Christ'
- Result: 'Nominee'
- Award: 'Best Supporting Actress'
- Film: 'The Passion of the Christ' (which is redundant as it's repeated and would normally be assumed to be a typo or a mistake)

Therefore, there is no term associated with Pasquale Preziosa that can be retrieved from this specific query. If you are looking for information about Pasquale Preziosa's term, it seems that this data is not available in the context of the Academy Award nominations for 'The Passion of the Christ'. You may need to consult a different data sou

## 2. Advanced Capability 2: Text-to-SQL with Query-Time Row Retrieval (along with Table Retrieval)

One problem in the previous example is that if the user asks a query that asks for "The Notorious BIG" but the artist is stored as "The Notorious B.I.G", then the generated SELECT statement will likely not return any matches.

We can alleviate this problem by fetching a small number of example rows per table. A naive option would be to just take the first k rows. Instead, we embed, index, and retrieve k relevant rows given the user query to give the text-to-SQL LLM the most contextually relevant information for SQL generation.

We now extend our workflow.

### Index Each Table

We embed/index the rows of each table, resulting in one index per table.

In [33]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index"
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: AcademyAwardActressNominations
Indexing rows in table: AirfieldLocationInfo
Indexing rows in table: AlbumChartPositions
Indexing rows in table: AwardsNomineesResults
Indexing rows in table: BBC_Radio_Financials
Indexing rows in table: BadBoyAlbums
Indexing rows in table: BoxingMatchResults
Indexing rows in table: Cancer_Gene_Functions
Indexing rows in table: ClimateMonthlyAverages
Indexing rows in table: ClubPerformanceStats
Indexing rows in table: DistrictPopulationDensity
Indexing rows in table: ExperimentTimeline
Indexing rows in table: FIFA_World_Cup_Group_Stage_Results
Indexing rows in table: FilmTitlesAndPerformances
Indexing rows in table: FootballSeasonResults
Indexing rows in table: GrammyAwardsNominees
Indexing rows in table: HorseRaceWinners
Indexing rows in table: KodachromeFilmHistory
Indexing rows in table: LanguagePhrases
Indexing rows in table: LocalGovernmentLeadershipTerms
Indexing rows in table: MPOfficeSuccessions
Indexing rows in table: Miss

### Define Expanded Table Parsing

We expand the capability of our table parsing to not only return the relevant table schemas, but also return relevant rows per table schema.

It now takes in both `table_schema_objs` (output of table retriever), but also the original `query_str` which will then be used for vector retrieval of relevant rows.

In [34]:
from llama_index.core.retrievers import SQLRetriever
from typing import List

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str,
    table_schema_objs: List[SQLTableSchema],
    verbose: bool = False,
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        if verbose:
            print(f"> Table Info: {table_info}")

        context_strs.append(table_info)
    return "\n\n".join(context_strs)

### Define Expanded Workflow

We re-use the workflow in section 1, but with an upgraded SQL parsing step after text-to-SQL generation.

It is very easy to subclass and extend an existing workflow, and customizing existing steps to be more advanced. Here we define a new worfklow that overrides the existing `retrieve_tables` step in order to return the relevant rows.

In [35]:
from llama_index.core.workflow import (
    Workflow,
    StartEvent,
    StopEvent,
    step,
    Context,
    Event,
)


class TextToSQLWorkflow2(TextToSQLWorkflow1):
    """Text-to-SQL Workflow that does query-time row AND table retrieval."""

    @step
    def retrieve_tables(
        self, ctx: Context, ev: StartEvent
    ) -> TableRetrieveEvent:
        """Retrieve tables."""
        table_schema_objs = self.obj_retriever.retrieve(ev.query)
        table_context_str = get_table_context_and_rows_str(
            ev.query, table_schema_objs, verbose=self._verbose
        )
        return TableRetrieveEvent(
            table_context_str=table_context_str, query=ev.query
        )

Since the overall sequence of steps is the same, the graph should look the same.

In [36]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    TextToSQLWorkflow2, filename="text_to_sql_table_retrieval.html"
)

text_to_sql_table_retrieval.html


### Run Some Queries

We can now ask about relevant entries even if it doesn't exactly match the entry in the database.

In [37]:
workflow2 = TextToSQLWorkflow2(
    obj_retriever,
    text2sql_prompt,
    sql_retriever,
    response_synthesis_prompt,
    llm,
    timeout=300,
    verbose=True,
)

In [38]:
response = await workflow2.run(
    query="What was the year that The Notorious BIG was signed to Bad Boy?"
)
print(str(response))

Running step retrieve_tables
> Table Info: Table 'BadBoyAlbums' has columns: Act (VARCHAR), Year_signed (INTEGER), _Albums_released_under_Bad_Boy (VARCHAR), . The table description is: List of artists signed to Bad Boy Records and the number of albums released under the label.
Here are some relevant example rows (values in the same order as columns above)
('The Notorious B.I.G', 1993, '5')
('The Hitmen', 1993, '–')

> Table Info: Table 'AlbumChartPositions' has columns: Year (INTEGER), Title (VARCHAR), Chart_Positions_UK (VARCHAR), Chart_Positions_US (VARCHAR), Chart_Positions_NL (VARCHAR), Comments (VARCHAR), . The table description is: Chart positions of albums by year, including UK, US, and NL rankings, and comments.
Here are some relevant example rows (values in the same order as columns above)
(1977, 'Novella', '–', '46', '–', '1977 (January in US, August in UK, as the band moved to the Warner Bros Music Group)')
(1978, 'A Song for All Seasons', '35', '58', '–', 'UK:Silver')

> Ta